In [1]:
!conda init bash
!source ~/.bashrc
!conda activate csnlp_project

no change     /Users/feichi/annaconda/anaconda3/condabin/conda
no change     /Users/feichi/annaconda/anaconda3/bin/conda
no change     /Users/feichi/annaconda/anaconda3/bin/conda-env
no change     /Users/feichi/annaconda/anaconda3/bin/activate
no change     /Users/feichi/annaconda/anaconda3/bin/deactivate
no change     /Users/feichi/annaconda/anaconda3/etc/profile.d/conda.sh
no change     /Users/feichi/annaconda/anaconda3/etc/fish/conf.d/conda.fish
no change     /Users/feichi/annaconda/anaconda3/shell/condabin/Conda.psm1
no change     /Users/feichi/annaconda/anaconda3/shell/condabin/conda-hook.ps1
no change     /Users/feichi/annaconda/anaconda3/lib/python3.8/site-packages/xontrib/conda.xsh
no change     /Users/feichi/annaconda/anaconda3/etc/profile.d/conda.csh
no change     /Users/feichi/.bash_profile
No action taken.
/Users/feichi/annaconda/anaconda3/envs/csnlp_project/bin/python: Error while finding module specification for 'virtualenvwrapper.hook_loader' (ModuleNotFoundError: No mod

In [2]:
import pandas as pd
import numpy as np
import string
import re
import os

In [ ]:
punctuation = string.punctuation+'“'+'”'+'‘'+'’'
# remove some special punctuation(specify in punctuation)        
def remove_punctuation(sent, punctuation = punctuation):
    no_punct=[words for words in sent if words not in punctuation]
    sent_wo_punct=''.join(no_punct)
    return sent_wo_punct

In [ ]:
# change to lower case
def lower_case(sent):
    sent_lower = sent.lower()
    return sent_lower

In [ ]:
# replace chinese symbol to english
def replace_symbol(sent):
    sent = sent.replace('“', '"')
    sent = sent.replace('”', '"')
    return sent

In [ ]:
# extract example from the list
def list_sent(row, remove_context = True):
    sents = [sent for sent in row['example'] if row['word'] in sent]
    return sents

In [ ]:
# split example and definition by \n and \r into lists
def sent_split(sent):
    sent1 = sent.split('\n')
    sent2 = []
    for x in sent1:
        sent2 = sent2+x.split('\r')
    sent3 = [x for x in sent2 if x != '']
    return sent3

In [ ]:
# combine definition list into a string
def definition_combine(row):
    list_def = row['definition']
    defi = ""
    for defi0 in list_def:
        defi+=defi0
    return defi

In [ ]:
# combine word and definition
def combine_word_def(row):
    combine = str(row['word']) + ' <sep> ' + str(row['definition'])
    return combine

In [ ]:
url_bracket = '[]'
data = pd.read_csv('data/urban_dictionary_new_10000.csv')
data = data.fillna('')
data[['definition', 'example']] = data[['definition', 'example']].applymap(lambda x: remove_punctuation(x, url_bracket))
data[['word', 'definition', 'example']] = data[['word', 'definition', 'example']].applymap(lower_case)
data[['definition', 'example']] = data[['definition', 'example']].applymap(lambda x: replace_symbol(x))
data[['definition', 'example']] = data[['definition', 'example']].applymap(lambda x: sent_split(x))
data[['example']] = data.apply(lambda x: list_sent(x), axis = 1)
data[['definition']] = data.apply(lambda x: definition_combine(x), axis = 1)

In [ ]:
df_new = pd.DataFrame(columns = ['word', 'definition', 'example'])
# split the example list
for i in range(len(data)):
    if i%1000 ==0:
        print(i)
    row = data.loc[i]
    if row['word'][0]>='A' and row['word'][0]<='z':
        df_temp = pd.DataFrame(columns = ['word', 'definition', 'example'])
        df_temp['example'] = row['example']
        df_temp['definition'] = row['definition']
        df_temp['word'] = row['word']
        df_new = pd.concat([df_new, df_temp], axis = 0)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000


In [ ]:
df_new = df_new[['word', 'definition', 'example']].sort_values(['word']).reset_index(drop = True)
df_new

In [ ]:
to_dir = 'data/'
name = 'data_cleaned_new.csv'
if not os.path.exists(to_dir):
    os.mkdir(to_dir)
df_new.to_csv(to_dir+name)

# arrange into fine-tune training style

In [ ]:
file_dir = 'data/data_cleaned_new.csv'
df_new = pd.read_csv(file_dir, index_col = 0)
df_combine = df_new
df_combine['combine'] = df_combine.apply(lambda x: combine_word_def(x), axis = 1)
df_combine = df_combine[["combine", "example"]]
df_combine.columns = ['source', 'target']

In [ ]:
to_dir = 'data/'
name = 'fine_tune_data_new.csv'
df_combine.to_csv(to_dir+name, encoding = 'utf-8')

# Preprocessing Normal Data